<a href="https://colab.research.google.com/github/anjunjin/PFE-ING3-IA/blob/branch1/LinkWithTwitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation de la library *python-dotenv*

In [4]:
!pip install python-dotenv

Importation des libraries

In [1]:
import tweepy
import pickle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from dotenv import dotenv_values
import requests
from IPython.display import JSON

Liaison au Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/PFE/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Récupération dans ***data*** des donnés nécessaires de tableauSites.csv

In [7]:
data = pd.read_csv(DATA_PATH+"tableauSites.csv", encoding='utf-8').drop(['Unnamed: 0', 'id'], axis=1).values.tolist()

Récupération des clés pour se connecter au compte Twitter Developers et utiliser son API

In [8]:
API_TOKEN = pd.read_csv(DATA_PATH+"Clés.csv", encoding='utf-8', sep=',')['Bearer Token'][0]
API_KEY = pd.read_csv(DATA_PATH+"Clés.csv", encoding='utf-8', sep=',')['API Key'][0]
API_SECRET = pd.read_csv(DATA_PATH+"Clés.csv", encoding='utf-8', sep=',')['API Key Secret'][0]

Authentifiation au compte Twitter Developers et récupération de l'API tweepy

In [9]:
auth = tweepy.AppAuthHandler(API_KEY, API_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

Création d'une classe TwitterUser contenant les fonctions de récupération de données (utilisateurs, tweets, ...)

In [10]:
    class TwitterUser():
        _screen_name = ""
        _id = ""
        _user_data = None
        followers = []
        following = []
        tweet = []
        like = []
        request_count = 0

        def __init__(self, sname = "", _json = {}):
            if (sname != ""):
                self._screen_name = sname
                self.get_info()
            if (_json != {}):
                self._json = _json

        def get_info(self):
            try:
                self._user_data = api.get_user(screen_name=self._screen_name)
            except Exception as e:
                print("Exception get_info")

        def get_all_data(self):
            self.get_tweets()
            # self.get_followers()
            # self.get_following()
            # self.get_like()

        def set_name(self, name):
            self._screen_name = name

        def set_id(self, _id):
            self._id = _id

        def get_followers(self):
            with tqdm(total=self._user_data.followers_count, desc="get_followers for " + self._user_data.screen_name) as pbar:
                for page in tweepy.Cursor(api.followers, screen_name=self._screen_name, count=200).pages():
                    self.followers.extend(page)
                    pbar.update(len(page))


        def get_following(self):
            with tqdm(total=self._user_data.friends_count, desc="get_following for " + self._user_data.screen_name) as pbar:
                for page in tweepy.Cursor(api.friends, screen_name=self._screen_name, count=200).pages():
                    self.following.extend(page)
                    pbar.update(len(page))

        # on ne peut prendre que les 3000 derniers
        def get_tweets(self):
            try:
                with tqdm(total=1000, desc="get_tweets for " + self._user_data.screen_name) as pbar:
                    for page in tweepy.Cursor(api.user_timeline, screen_name=self._screen_name, count=200).pages():
                        self.tweet.extend(page)
                        pbar.update(len(page))
            except Exception as e:
                print("Exception get_tweets")

        # on ne peut prendre que les 3000 derniers
        def get_like(self):
            if self._user_data.protected:
                print("can get like from " + self._user_data.screen_name + ", the account is protected")
            else:
                with tqdm(total=3000, desc="get_likes for " + self._user_data.screen_name) as pbar:
                    for page in tweepy.Cursor(api.favorites, screen_name=self._screen_name, count=200).pages():
                        self.like.extend(page)
                        pbar.update(len(page))


Division de la liste des site en plusieurs sous liste pour ne pas exéder la RAM alloué par Google Colab 

In [11]:
def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]

dataA, dataB = split_list(data)
data1, data2 = split_list(dataA)
data3, data4 = split_list(dataB)

Ajout de tous les tweets dans 4 tableaux qui vont contenir les données importantes de chaque tweet. Chaque tableau contient un quart des données de ***data***. Les tableaux sont ensuite sauvegardé au format pkl sur le Drive.

Pour obtenir chaque morceau il faut relancer la session (ce qui libère la RAM) et exécuter le bloc de commande contenant le morceau voulu.

Création et sauvegarde du tableau 1

In [ ]:
list_tweet = []

for site in data1:
  user = TwitterUser(sname= site[2])
  user.get_all_data()
  for tweet in user.tweet:
    list_tweet.append([tweet.user.id, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, site[1]])

res = pd.DataFrame(list_tweet, columns=['user_id', 'id_tweet', 'tweet', 'retweet_count', 'favorite_count', 'label_liability'])
res.to_pickle(DATA_PATH+"quartier1.pkl")

Création et sauvegarde du tableau 2

In [ ]:
list_tweet = []

for site in data2:
  user = TwitterUser(sname= site[2])
  user.get_all_data()
  for tweet in user.tweet:
    list_tweet.append([tweet.user.id, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, site[1]])

res = pd.DataFrame(list_tweet, columns=['user_id', 'id_tweet', 'tweet', 'retweet_count', 'favorite_count', 'label_liability'])
res.to_pickle(DATA_PATH+"quartier2.pkl")

Création et sauvegarde du tableau 3

In [ ]:
list_tweet = []

for site in data2:
  user = TwitterUser(sname= site[2])
  user.get_all_data()
  for tweet in user.tweet:
    list_tweet.append([tweet.user.id, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, site[1]])

res = pd.DataFrame(list_tweet, columns=['user_id', 'id_tweet', 'tweet', 'retweet_count', 'favorite_count', 'label_liability'])
res.to_pickle(DATA_PATH+"quartier3.pkl")

Création et sauvegarde du tableau 4

In [ ]:
list_tweet = []

for site in data2:
  user = TwitterUser(sname= site[2])
  user.get_all_data()
  for tweet in user.tweet:
    list_tweet.append([tweet.user.id, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, site[1]])

res = pd.DataFrame(list_tweet, columns=['user_id', 'id_tweet', 'tweet', 'retweet_count', 'favorite_count', 'label_liability'])
res.to_pickle(DATA_PATH+"quartier4.pkl")

Après avoir obtenu les 4 tableaux en ayant relancé 4 sessions, une pour la création de chaque tableau, on récupère les 4 tableaux.

In [4]:
tableauTweets1 = pd.read_pickle(DATA_PATH+"quartier1.pkl")
tableauTweets2 = pd.read_pickle(DATA_PATH+"quartier2.pkl")
tableauTweets3 = pd.read_pickle(DATA_PATH+"quartier3.pkl")
tableauTweets4 = pd.read_pickle(DATA_PATH+"quartier4.pkl")

Concaténation des 4 tableaux en un seul et sauvegarde dans un seul fichier

In [ ]:
tableauTweets = pd.concat([tableauTweets1,tableauTweets2,tableauTweets3,tableauTweets4])
tableauTweets.reset_index(drop=True)
tableauTweets.to_pickle(DATA_PATH+"tableauTweets.pkl")

Réinitialisation du fichier PauchBas.pickle

In [ ]:
import pickle

with open(DATA_PATH+"PaucBas.pickle", "wb") as obj_file:
    pickle.dump([], obj_file, -1)

Sauvegarde des données reçu pour chaque utilisateur Twitter recherché

In [ ]:
# with open(DATA_PATH+"PaucBas.pickle", "ab") as obj_file:
#   for site in data1:
#     user = TwitterUser(sname= site[2])
#     user.get_all_data()
#     pickle.dump([user], obj_file, pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open(DATA_PATH+"PaucBas.pickle", "ab") as obj_file:
#   for site in data2:
#     user = TwitterUser(sname= site[2])
#     user.get_all_data()
#     pickle.dump([user], obj_file, pickle.HIGHEST_PROTOCOL)